# Imports

In [22]:
import gymnasium as gym
from gymnasium import spaces
# from stable_baselines3 import PPO
# from scipy.optimize import minimize, Bounds, LinearConstraint
import plotly.graph_objs as go
import pandas as pd
import requests
import numpy as np
import yfinance as yf
import matplotlib
%matplotlib inline
import random
# import cvxpy as cp
# import matplotlib.pyplot as plt
import datetime as dt
# from prophet import Prophet
from sklearn.metrics import r2_score, mean_absolute_error
# from stable_baselines3.common.vec_env import DummyVecEnv
# import torch
from flipside import Flipside

import os
from dotenv import load_dotenv

import datetime as dt
from datetime import timedelta

from sklearn.linear_model import LinearRegression

In [23]:
os.chdir('..')

In [24]:
from python_scripts.utils import flipside_api_results, set_random_seed, to_time, clean_prices
from sql_scripts.queries import prices, volume 

In [25]:
os.chdir('notebooks')

# Environment Variables

In [27]:
set_random_seed(20)

In [28]:
load_dotenv()
flipside_api_key = os.getenv("FLIPSIDE_API_KEY")

# Data Collection

## BTC Volume

In [31]:
btc = yf.Ticker('BTC-USD')
btc_df = btc.history(period='max')
btc_df = btc_df['Volume'].to_frame('BTC Volume')
btc_df

,BTC Volume
Date,
2014-09-17 00:00:00+00:00,21056800
2014-09-18 00:00:00+00:00,34483200
2014-09-19 00:00:00+00:00,37919700
2014-09-20 00:00:00+00:00,36863600
2014-09-21 00:00:00+00:00,26580100
...,...
2024-08-19 00:00:00+00:00,25911207712
2024-08-20 00:00:00+00:00,31613400008
2024-08-21 00:00:00+00:00,32731154072


## DEX Volume & Asset Prices (Onchain Data)

In [33]:
def pull_data(api=False):
    prices_path = '../data/prices.csv'
    volume_path = '../data/volume.csv'

    if api == True:
        prices_df = flipside_api_results(prices, flipside_api_key)
        prices_df.to_csv(prices_path, index=False)
        volume_df = flipside_api_results(prices, flipside_api_key)
        volume_df.to_csv(volume_path, index=False)
    else:
        prices_df = pd.read_csv(prices_path)
        volume_df = pd.read_csv(volume_path)

    return prices_df, volume_df 

prices_df = flipside_api_results(prices, flipside_api_key)
prices_path = '../data/prices.csv'
prices_df.to_csv(prices_path, index=False)

volume_df = flipside_api_results(volume, flipside_api_key)
volume_path = '../data/volume.csv'
volume_df.to_csv(volume_path, index=False)

In [36]:
prices_df, volume_df = pull_data(api=False)

In [37]:
clean_prices_df = clean_prices(prices_df)
clean_prices_df = to_time(clean_prices_df)
if '__row_index' in clean_prices_df.columns:
    clean_prices_df.drop(columns=['__row_index'], inplace=True)
clean_prices_df

clean_prices_df

DatetimeIndex(['2018-02-14 01:00:00+00:00', '2018-02-14 02:00:00+00:00',
               '2018-02-14 03:00:00+00:00', '2018-02-14 04:00:00+00:00',
               '2018-02-14 05:00:00+00:00', '2018-02-14 06:00:00+00:00',
               '2018-02-14 07:00:00+00:00', '2018-02-14 08:00:00+00:00',
               '2018-02-14 09:00:00+00:00', '2018-02-14 10:00:00+00:00',
               ...
               '2024-08-23 12:00:00+00:00', '2024-08-23 13:00:00+00:00',
               '2024-08-23 14:00:00+00:00', '2024-08-23 15:00:00+00:00',
               '2024-08-23 16:00:00+00:00', '2024-08-23 17:00:00+00:00',
               '2024-08-23 18:00:00+00:00', '2024-08-23 19:00:00+00:00',
               '2024-08-23 20:00:00+00:00', '2024-08-23 21:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='dt', length=57189, freq=None)


,BTC Price,ETH Price
dt,,
2018-02-14 01:00:00+00:00,NaN,839.535
2018-02-14 02:00:00+00:00,NaN,839.535
2018-02-14 03:00:00+00:00,NaN,839.535
2018-02-14 04:00:00+00:00,NaN,839.535
2018-02-14 05:00:00+00:00,NaN,839.535
...,...,...
2024-08-23 17:00:00+00:00,61413.0,2674.400
2024-08-23 18:00:00+00:00,61564.0,2675.120
2024-08-23 19:00:00+00:00,62716.0,2727.840


In [38]:
volume_df = to_time(volume_df)
volume_df.rename(columns={"volume":"DEX Volume"}, inplace=True)
if '__row_index' in volume_df.columns:
    volume_df.drop(columns=['__row_index'], inplace=True)
volume_df

DatetimeIndex(['2024-08-23 20:00:00+00:00', '2024-08-23 19:00:00+00:00',
               '2024-08-23 18:00:00+00:00', '2024-08-23 17:00:00+00:00',
               '2024-08-23 16:00:00+00:00', '2024-08-23 15:00:00+00:00',
               '2024-08-23 14:00:00+00:00', '2024-08-23 13:00:00+00:00',
               '2024-08-23 12:00:00+00:00', '2024-08-23 11:00:00+00:00',
               ...
               '2019-09-27 03:00:00+00:00', '2019-09-27 00:00:00+00:00',
               '2019-09-26 22:00:00+00:00', '2019-09-26 21:00:00+00:00',
               '2019-09-26 19:00:00+00:00', '2019-09-26 18:00:00+00:00',
               '2019-09-26 17:00:00+00:00', '2019-09-26 14:00:00+00:00',
               '2019-09-26 13:00:00+00:00', '2019-09-26 10:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='dt', length=42228, freq=None)


,DEX Volume
dt,
2024-08-23 20:00:00+00:00,5.588734e+07
2024-08-23 19:00:00+00:00,2.136648e+08
2024-08-23 18:00:00+00:00,3.260078e+08
2024-08-23 17:00:00+00:00,2.633585e+08
2024-08-23 16:00:00+00:00,1.448834e+08
...,...
2019-09-26 18:00:00+00:00,1.400500e+03
2019-09-26 17:00:00+00:00,1.549980e+03
2019-09-26 14:00:00+00:00,1.137598e+04


In [53]:
## RWA.xyz Data

In [55]:
rwa_commodities_path = '../data/rwa_commodities.csv'
rwa_commodities_df = pd.read_csv(rwa_commodities_path)
rwa_commodities_df.head()

,Timestamp,Date,Measure,PAXG,XAUT,TXAU,TXAG,TXPT,VNXAU,WTGOLD
0,1568592000000,2019-09-16,Circulating Market Value (Dollar),3621924.08,NaN,NaN,NaN,NaN,NaN,NaN
1,1568678400000,2019-09-17,Circulating Market Value (Dollar),3849525.39,NaN,NaN,NaN,NaN,NaN,NaN
2,1568764800000,2019-09-18,Circulating Market Value (Dollar),3634625.66,NaN,NaN,NaN,NaN,NaN,NaN
3,1568851200000,2019-09-19,Circulating Market Value (Dollar),3624969.79,NaN,NaN,NaN,NaN,NaN,NaN
4,1568937600000,2019-09-20,Circulating Market Value (Dollar),3660449.60,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
rwa_bonds_path = '../data/rwa_global_bonds.csv'
rwa_bonds_df = pd.read_csv(rwa_bonds_path)
rwa_bonds_df.head()

,Timestamp,Date,Measure,XEVT,bERNX,bHIGH,bC3M
0,1695772800000,2023-09-27,Total Asset Value (Dollar),NaN,NaN,2251003.06,8642928.39
1,1695859200000,2023-09-28,Total Asset Value (Dollar),NaN,NaN,2251003.06,8642928.39
2,1695945600000,2023-09-29,Total Asset Value (Dollar),NaN,NaN,2262179.70,8698505.10
3,1696032000000,2023-09-30,Total Asset Value (Dollar),NaN,NaN,2273442.77,8711950.53
4,1696118400000,2023-10-01,Total Asset Value (Dollar),NaN,NaN,2273442.77,8711950.53


In [59]:
rwa_credit_path = '../data/rwa_private_credit.csv'
rwa_credit_df = pd.read_csv(rwa_credit_path)
rwa_credit_df.head()

,Timestamp,Date,Measure,Centrifuge,Maple,Credix,TrueFi,Figure,Goldfinch
0,1602979200000,2020-10-18,Outstanding Capital (Dollar),NaN,NaN,NaN,NaN,NaN,NaN
1,1603065600000,2020-10-19,Outstanding Capital (Dollar),NaN,NaN,NaN,NaN,NaN,NaN
2,1603152000000,2020-10-20,Outstanding Capital (Dollar),NaN,NaN,NaN,NaN,NaN,NaN
3,1603238400000,2020-10-21,Outstanding Capital (Dollar),NaN,NaN,NaN,NaN,NaN,NaN
4,1603324800000,2020-10-22,Outstanding Capital (Dollar),53107.91,NaN,NaN,NaN,NaN,NaN


In [61]:
rwa_loans_path = '../data/rwa_private_loans.csv'
rwa_loans_df = pd.read_csv(rwa_loans_path)
rwa_loans_df.head()

,loan_id,pool_id,protocol_id,protocol_name,network_id,network_name,base_asset_id,base_asset_ticker,loan_type,amortization_type,...,outstanding_principal_token,principal_paid_dollar,funded_assets_token,funded_assets_dollar,base_interest_rate,management_fee_rate,late_fee_interest_rate,funding_open_timestamp,term_start_timestamp,term_end_timestamp
0,0xda8f7941192590408dce701a60fb3892455669ce,0xfe119e9c24ab79f1bdd5dd884b86ceea2ee75d92,3,Maple,1,Ethereum,4.0,USDC,open_term,bullet,...,19854230.66,32985931.71,52842207.26,52838931.63,4.87,NaN,0.0,2024-08-22T15:09:11,2024-08-22T15:09:11,NaN
1,0xf6950f28353ca676100c2a92dd360dea16a213ce,0x6f6c8013f639979c84b756c7fc1500eb5af18dc4,3,Maple,1,Ethereum,4.0,USDC,fixed_term,bullet,...,15000000.00,0.00,15000000.00,14999070.17,12.00,NaN,0.0,2022-09-21T09:22:59,2022-09-21T09:23:59,NaN
2,0x418749e294cabce5a714efccc22a8aade6f9db57,0x8481a6ebaf5c7dabc3f7e09e44a89531fd31f822,1,Goldfinch,1,Ethereum,4.0,USDC,term_loan_fixed,bullet,...,11812267.21,0.06,11812267.27,11811535.04,10.00,0.0,0.0,2022-02-08T16:00:00,2022-02-21T16:13:56,2025-02-20T16:13:56
3,0x29356f80d6016583c03991cda7dd42259517c005,0xe9d33286f0e37f517b1204aa6da085564414996d,3,Maple,1,Ethereum,4.0,USDC,open_term,bullet,...,11294289.88,3156011.01,14450496.54,14449600.77,16.20,NaN,0.0,2024-07-22T18:05:59,2024-07-22T18:05:59,NaN
4,0x2466f80f4743825fff87e07e43a69b4d654db04f,0xc1dd3f011290f212227170f0d02f511ebf57e433,3,Maple,1,Ethereum,4.0,USDC,open_term,bullet,...,11000000.00,0.00,11000000.00,10999318.12,9.25,NaN,0.0,2024-07-18T23:19:59,2024-07-18T23:19:59,NaN


In [63]:
rwa_stocks_path = '../data/rwa_stocks.csv'
rwa_stocks_df = pd.read_csv(rwa_stocks_path)
rwa_stocks_df.head()

,Timestamp,Date,Measure,bCSPX,bCOIN
0,1695772800000,2023-09-27,Circulating Market Value (Dollar),2508015.92,NaN
1,1695859200000,2023-09-28,Circulating Market Value (Dollar),2508015.92,NaN
2,1695945600000,2023-09-29,Circulating Market Value (Dollar),2505215.92,NaN
3,1696032000000,2023-09-30,Circulating Market Value (Dollar),2534560.03,NaN
4,1696118400000,2023-10-01,Circulating Market Value (Dollar),2525935.99,NaN


In [65]:
rwa_tbills_path = '../data/rwa_treasuries.csv'
rwa_tbills_df = pd.read_csv(rwa_tbills_path)
rwa_tbills_df.head()

,Timestamp,Date,Measure,TBILL,MPLcashUSDC,MPLcashUSDC-SPL,MPLcashUSDT,bIBTA,bIB01,TBONDS13,...,WTTSX,TIPSX,WTGXX,USTB,BUIDL,TBV,XTBT,mTBILL,bZPR1,USFR.d
0,1632182400000,2021-09-21,Total Asset Value (Dollar),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1632268800000,2021-09-22,Total Asset Value (Dollar),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1632355200000,2021-09-23,Total Asset Value (Dollar),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1632441600000,2021-09-24,Total Asset Value (Dollar),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1632528000000,2021-09-25,Total Asset Value (Dollar),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Feature Engineering